In [1]:
#embedding 테스트 - 완료
##one-hot으로 할 필요가 없음.
##embedding_size도 마음대로 결정할 수 있음.
##random init으로 해도됨.
##input의 embedding을 training할수도 있고 안할수도 있음
##Y는 embedding 할 필요 없음. 그냥 target class shape만 맞춰서 loss를 구하면 됨

#file에서 읽어와서 input으로 넣어보기 - 완료
##이미 index화 되어 있는 파일 읽어오기 - 완료
##dictionary file로부터 dic 생성 - 완료
##embedding 활용 - 완료

#saver 적용 - 완료

#tensorboard 적용

#padding 적용

#batch 적용

#bidirectional rnn 적용

from __future__ import print_function

import tensorflow as tf
import numpy as np
import pprint as pp
import os

from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

In [2]:
checkpoint_path = 'data/dev/ckpt/'

def read_sentences_from_file(file_path):
    sentence_file = open(file_path, 'r')
    sentences = []
    while True:
        line = sentence_file.readline()
        if not line:
            break;
        sentences.append(line.replace('\n',''))
    sentence_file.close()
    return sentences

def read_sentence_indexes_from_file(file_path):
    sentence_index_file = open(file_path, 'r')
    sentence_indexes = []
    while True:
        line = sentence_index_file.readline()
        if not line:
            break;
        sentence_indexes.append([int(c) for c in line.replace('\n','').split()])
    sentence_index_file.close()
    return sentence_indexes

def char_tokenizer(sentence): #sentence는 byte로 들어옴
    sentence = sentence.replace('<english>','a').replace('<number>','0')
    return [char for char in sentence] #그 후 최종 다시 byte로 변환한다

def create_dic(sentences, save_file_path):
    dic = []
    file = open(save_file_path, 'w+')
    for sentence in sentences:
        tokens = char_tokenizer(sentence)
        for token in tokens:
            if not (token in dic):
                dic.append(token)
                file.write(token+'\n')
    file.close
    return dic
    
def load_dic(dic_file_path):
    dic_file = open(dic_file_path, 'r')
    rev_vocab_dic = []
    while True:
        line = dic_file.readline()
        if not line:
            break;
        rev_vocab_dic.append(line.replace('\n',''))
    vocab_dic = {w: i for i, w in enumerate(rev_vocab_dic)}
    return vocab_dic, rev_vocab_dic

def sentences_to_indexes(sentences, vocab_dic):
    indexes = []
    for sentence in sentences:
        tokens = char_tokenizer(sentence)
        indexes.append([vocab_dic[char] for char in tokens])
    return indexes

In [3]:
##직접 dic을 구축하고, index를 만드려면 아래의 코드를 사용
# chosung_sentences = read_sentences_from_file('data/dev/namuwiki_chosung_dev.txt')
# target_sentences = read_sentences_from_file('data/dev/namuwiki_sentences_dev.txt')

# _ = create_dic(chosung_sentences, 'data/dev/chosung_dic_dev')
# _ = create_dic(target_sentences, 'data/dev/target_dic_dev')

# chosung_vocab_dic, chosung_rev_vocab_dic = load_dic('data/dev/chosung_dic_dev')
# target_vocab_dic, target_rev_vocab_dic = load_dic('data/dev/target_dic_dev')

# chosung_sentence_indexes = sentences_to_indexes(chosung_sentences, chosung_vocab_dic)
# target_sentence_indexes = sentences_to_indexes(target_sentences, target_vocab_dic)

In [4]:
##이미 만들어진 dic과 index를 사용하려면 아래의 코드를 사용
chosung_sentence_indexes =  read_sentence_indexes_from_file('data/dev/namuwiki_chosung_dev.txt.ids10000')
target_sentence_indexes =  read_sentence_indexes_from_file('data/dev/namuwiki_sentences_dev.txt.ids10000')
chosung_vocab_dic, chosung_rev_vocab_dic = load_dic('data/dev/vocab_dev.from')
target_vocab_dic, target_rev_vocab_dic = load_dic('data/dev/vocab_dev.to')

In [5]:
data_dim = len(target_vocab_dic)
hidden_size = 30
num_classes = 3000
embedding_size = 30
sequence_length = 50
learning_rate = 0.05
batch_size = len(chosung_sentence_indexes)

In [6]:
dataX = []
dataY = []
output_weights = []
sequence_length_arr = []

for i in range(batch_size):
    output_weight = np.ones(sequence_length, dtype=np.float32)
    chosung_indexes = list(chosung_sentence_indexes[i])
    target_indexes = list(target_sentence_indexes[i])
    if len(chosung_indexes) > sequence_length:
        chosung_indexes = chosung_indexes[:sequence_length]
        target_indexes = target_indexes[:sequence_length]
        sequence_length_arr.append(sequence_length)
    else:
        sequence_length_arr.append(len(chosung_indexes))
        pad_size = sequence_length - len(chosung_indexes)
        chosung_indexes += [0]*pad_size #padding
        target_indexes += [0]*pad_size #padding
        for j in range(pad_size):
            output_weight[sequence_length-1-j] = 0.0 #padding인 경우 weight를 0으로 초기화
    dataX.append(chosung_indexes) 
    dataY.append(target_indexes)
    output_weights.append(output_weight)
print(sequence_length_arr)
    

[23, 23, 37, 45, 41, 23, 23, 23, 36, 50]


In [7]:
X = tf.placeholder(tf.int32, [None, sequence_length], name="chosung_sentences")
Y = tf.placeholder(tf.int32, [None, sequence_length], name="target_sentences")
weights = tf.placeholder(tf.float32, [None, sequence_length], name="seqence_weights")
output_sequence_size = tf.placeholder(tf.int32, [None], name="output_sequence_size")

In [8]:
print(num_classes)
embeddings = tf.Variable(
    tf.random_uniform([len(chosung_vocab_dic), embedding_size], -1.0, 1.0), trainable=False, name="chosung_embeddings")
embedded_input = tf.nn.embedding_lookup(embeddings, X, name="embedded_chosung_sentences")
print(embedded_input)

3000
Tensor("embedded_chosung_sentences:0", shape=(?, 50, 30), dtype=float32)


In [9]:
# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, embedded_input, dtype=tf.float32, sequence_length=output_sequence_size)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size], name="reshaped_rnn_output")
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes], name="predictions")

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights, name="sequence_loss")
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [10]:
sess = tf.Session()
# saver = tf.train.Saver(tf.global_variables())

# ckpt = tf.train.get_checkpoint_state(checkpoint_path)
# if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
#     print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
#     saver.restore(sess, ckpt.model_checkpoint_path)
# else:
#     print("Created model with fresh parameters.")
#     sess.run(tf.global_variables_initializer())
sess.run(tf.global_variables_initializer())

In [11]:
for i in range(num_classes):
    target_rev_vocab_dic.append('_')

In [12]:
print(sequence_length_arr)

[23, 23, 37, 45, 41, 23, 23, 23, 36, 50]


In [13]:
for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY, weights: output_weights, output_sequence_size: sequence_length_arr})
    if i % 50 == 0:
        print('step:',i,' loss:',l)
        results = sess.run(outputs, feed_dict={X: dataX, output_sequence_size: sequence_length_arr})
        for sentence_result in results:
            index = np.argmax(sentence_result, axis=1)
            print(''.join([target_rev_vocab_dic[t] for t in index]))
        print('----------')
#         checkpoint_file_path = os.path.join(checkpoint_path, "chosung_rnn.ckpt")
#         saver.save(sess, checkpoint_file_path)
#         print('saved checkpoint')


step: 0  loss: 8.01047
소소소소소소소소소소소소소소소소소소소소소소소                           
소소소소소소소소소소소소소소소소소소소소소소소                           
소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소             
소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소     
소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소         
소소소소소소소소소소소소소소소소소소소소소소소                           
소소소소소소소소소소소소소소소소소소소소소소소                           
소소소소소소소소소소소소소소소소소소소소소소소                           
소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소              
크크삼소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소소
----------
step: 50  loss: 3.17894
                       여여여여여여여여여여여여여여여여여여여여여여여여여여여
                       여여여여여여여여여여여여여여여여여여여여여여여여여여여
                                     여여여여여여여여여여여여여
                                             여여여여여
                                         여여여여여여여여여
                       여여여여여여여여여여여여여여여여여여여여여여여여여여여
                       여여여여여여여여여여여여여여여여여여여여여여여여여여여
                       여여여여여여여여여여여여여여여여여여여여여여여여여여여
                        

In [ ]:
print(len(results)) #batch size
print(len(results[0])) #sequence length
print(len(results[0][0])) #vocab_size

print(batch_size)
print(sequence_length)
print(len(target_vocab_dic))


In [ ]:
#embedding 연습!!
import tensorflow as tf

embeddings = tf.Variable(
    tf.random_uniform([num_classes, 50], -1.0, 1.0))

train_inputs = tf.placeholder(tf.int32, shape=[3,2])
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
ems, inputs,embd = sess.run([embeddings,train_inputs,embed], feed_dict={train_inputs:[[1,2],[2,3],[3,4]]})

for i in ems:
    print(i)
print('-----------')
print(inputs)
print('-----------')
print(embd)